In [1]:
import pickle
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.models import Sequential
from numpy.linalg import norm
from sklearn.neighbors import NearestNeighbors
import cv2

In [2]:
import joblib
import numpy as np

features_list = joblib.load("image_features_embedding.pkl")
img_files_list = joblib.load("img_files.pkl")

print(np.array(features_list).shape)


(361, 2048)


In [3]:


model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
model.trainable = False

model = Sequential([model, GlobalMaxPooling2D()])


94765736/94765736 [==============================] - 95s 1us/step


In [8]:

img = image.load_img('Shoes.jpg',target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()

1/1 [==============================] - 0s 269ms/step


In [9]:
normalized_result = result / norm(result)

neighbors = NearestNeighbors(n_neighbors=6,algorithm='brute',metric='euclidean')
neighbors.fit(features_list)

distances,indices = neighbors.kneighbors([normalized_result])

print(indices)

[[ 48  78 180 337 339  87]]


In [10]:
for file in indices[0][1:6]:
    print(img_files_list[file])


images/images/10093.jpg
images/images/10089.jpg
images/images/10161.jpg
images/images/10081.jpg
images/images/10111.jpg


In [12]:
import cv2
import matplotlib.pyplot as plt

# Load and resize the image
tmp_img = cv2.imread(img_files_list[file])
#tmp_img = cv2.resize(tmp_img, (200, 200))

# Display the image using matplotlib
plt.imshow(cv2.cvtColor(tmp_img, cv2.COLOR_BGR2RGB))
plt.axis('off')  # Turn off axis labels and ticks
plt.show()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
